# VIE Airport Türen

In [1]:
from pandas import read_excel, concat, to_numeric
from datetime import datetime as dt
from viedoors.util import level_mapper, clean_df, flt_object_mapper



int_mapper = lambda x: int(x)
x_mapper = lambda x: True if x=="X" else False

date_mapper = lambda x: dt.strptime(x, "%d.%m.%y")
cad_number_mapper = lambda x: f"{int(x):02}"

## NPA manuelle Mitarbeiterliste

Bei der Liste „Durch Mitarbeiter aufgenommene Anlagenliste (Dateinamen „NPA_Tür Aufnahme Obj. 420“)“ ist der Feuerwiderstand jenes Element, welches als Grundlage für den Vergleich mit den anderen Listen herangezogen werden soll. Folgende relevanten Elemente müssen mit den anderen Listen verglichen und aufgelistet werden

* AKS-Nummer Plan (setzt sich zusammen aus Objekt/Ebene/Bauteil/Aks Plan) – Bsp. 103 00 A0220.01
* AKS-Nummer real (setzt sich zusammen aus Objekt/Ebene/Bauteil/Aks real) – Bsp. 103 00 0A0220.01
* Schlossernummer (Bsp. 103/0/1)
* Alte Türnummer (Bsp. BW102)
* Fluchttürnummer (Spalte Q) neu

In [2]:
file_name = "NPA_Tuer Aufnahme Obj420.xlsx"

npa_data = read_excel(
    io=f"data/{file_name}"
)

npa_data["Ebene"] = npa_data["Ebene"].map(level_mapper)
npa_data["aks_soll"] = npa_data["Objekt"].astype(str) + " " + npa_data["Ebene"].astype(str) + "" + npa_data["Bauteil"].astype(str) + "" + npa_data["AKS Plan"].astype(str)
npa_data["aks_ist"] = npa_data["Objekt"].astype(str) + " " + npa_data["Ebene"].astype(str) + "" + npa_data["Bauteil"].astype(str) + "" + npa_data["AKS real"].astype(str)

npa_data = clean_df(npa_data, "npa")

npa_data.head()

,npa__objekt,npa__ebene,npa__bauteil,npa__aks_plan,npa__aks_real,npa__alte_tuernummer_,npa__schlossernummer,npa__hoehe_[cm],npa__breite_[cm],npa__zylinder_[z/b/wc/s]_,...,npa__fluchtwegs-tuer_nr,npa__panik_[d/_st k],npa__nottaster_[ja/nein],npa__offenhalte-magnet_[ja/nein],npa__taster_fuer_schliessung_[ja/nein],npa__vorsicherung_[ja/nein],npa__sz_-_magnet_[ja/nein],npa__bemerkung,npa__aks_soll,npa__aks_ist
0,420,K1,A,2205.01,keine Nr.,5K2,063 / K / 48,200,80,nein,...,keine Nr.,D / D,nein,ja,ja,nein,nein,im Plan rot,420 K1A2205.01,420 K1Akeine Nr.
1,420,K1,A,2506.01,keine Nr.,5K3,063 / K / 47,200,80,nein,...,keine Nr.,D / D,nein,ja,ja,nein,nein,im Plan rot,420 K1A2506.01,420 K1Akeine Nr.
2,420,K1,A,2308.01,keine Nr.,01A,keine Nr.,243,112,nein,...,keine Nr.,D / D,nein,nein,nein,nein,nein,NaN,420 K1A2308.01,420 K1Akeine Nr.
3,420,K1,A,2108.01,keine Nr.,5K1,063 / K / 109,200,80,nein,...,keine Nr.,D / D,nein,ja,ja,nein,nein,im Plan rot,420 K1A2108.01,420 K1Akeine Nr.
4,420,K1,A,1305.02,keine Nr.,3K1,063 / K / 149,200,80,nein,...,keine Nr.,D / D,nein,ja,ja,nein,nein,im Plan rot,420 K1A1305.02,420 K1Akeine Nr.


## Haltemagnete Schrack

In [3]:
file_name = "Schrack_HM_Obj420.xls"

hm_data = read_excel(
    io=f"data/{file_name}",
    sheet_name="HALTEMAGNETE"
)
hm_data["GESCH."] = hm_data["GESCH."].astype(str).map(level_mapper)
hm_data["aks"] = hm_data["TÜRNR(AKS)"]

hm_data = clean_df(hm_data, "hm")

hm_data.head(10)

,hm__obj,hm__gesch,hm__tuernr(aks),hm__tuernralt,hm__bstuerealt,hm__raumnr,hm__aks_raum,hm__uz6/steu,hm__kreis,hm__lfd _nr,hm__info,hm__aks
0,420.0,00,420 EG0A0106.02,004,420/0/3/6,001,0A0106,5.0,1.0,1.0,NaN,420 EG0A0106.02
1,NaN,##,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,420.0,00,420 EG0A0803.02,040D,420/0/14/6,040D,0A0803,6.0,2.0,2.0,NaN,420 EG0A0803.02
3,420.0,00,420 EG0A0602.01,040A,420/0/15/6,040A,0A0602,6.0,2.0,3.0,NaN,420 EG0A0602.01
4,420.0,00,420 EG0A0406.01,008,420/0/22/6,004,0A0406,6.0,2.0,4.0,NaN,420 EG0A0406.01
5,420.0,00,420 EG A0807.02,009,420/0/10,009,0A0807,6.0,2.0,5.0,NaN,420 EG A0807.02
6,NaN,##,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,420.0,00,420 EG0A2207.01,017,420/0/40/6,017,0A2207,7.0,3.0,1.0,Spannungs-\nunterbrechung\nvon 17.00-06.30 übe...,420 EG0A2207.01
8,NaN,##,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,420.0,00,420 EG0A1206.01,018,420/0/45/6,008,0A1206,8.0,4.0,1.0,Spannungs-\nunterbrechung\nvon 17.00-06.30 übe...,420 EG0A1206.01


## Brandschutztechnik (BST) Sisando

In [4]:
file_name = "Sisando_BST_Obj420.xlsx"

bst_data = read_excel(
    io=f"data/{file_name}",
    sheet_name="Sheet1",
    parse_dates=[9]
)

bst_data["Ebene"] = bst_data["Ebene"].astype(str).map(level_mapper)
bst_data["aks"] = bst_data["Name"].astype(str) + " " + bst_data["Ebene"] + bst_data["Raum2"]

bst_data = clean_df(bst_data, "bst")

bst_data.head(5)

,bst__name,bst__ebene,bst__nummer,bst__statustext,bst__status,bst__class,bst__raum2,bst__tow todanmerkung,bst__tow_tod_lastdurchfuehrung,bst__tow_tod_lastpnl_nme,bst__pnl tow tod_lastf_name,bst__exf01,bst__haltemechanik,bst__hersteller,bst__klassifikation,bst__aks
0,420,00,A0201.01,OK,1,Drehflügel #1,A0201,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,NaN,Peneder,T30,420 00A0201
1,420,00,A0203.01,OK,1,Drehflügel #2,A0203,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,1.0,NaN,Peneder,EI2-30-C,420 00A0203
2,420,00,A0803.01,OK,1,Drehflügel #1,A0803,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,1.0,NaN,Peneder,EI2-30-C,420 00A0803
3,420,00,A0803.02,HINWEIS,21,Drehflügel #2,A0803,STATUS OK_x001D_H000 Freilauftürschließer hält...,2024-10-28,peD,David PESCHEK,1.0,NaN,Fritscher,T30,420 00A0803
4,420,00,A0807.03,OK,1,Drehflügel #2,A0807,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,NaN,Fritscher,T30,420 00A0807


## Feuerleittechnik (FLT) Sisando

In [5]:
file_name = "Sisando_FLT_Obj420.xlsx"

columns = [
    "Laufende Nummer",
    "Plan Nr.",
    "Bauteil",
    "Ebene",
    "Modul",
    "Raumnummer",
    "Türnummer",
    "Flucht möglich Ja",
    "Flucht möglich Nein",
    "Blindzylinder/Rosette/Abdeckung",
    "Zylinder",
    "Panikschloss",
    "Versperrt",
    "Versperrbar Ja",
    "Versperrbar Nein",
    "Nottaster Ja",
    "Nottaster Nein",
    "Mängel Art",
    "Mängel erl",
    "leer 1",
    "leer 2",
    "leer 3"
]

flt_data = read_excel(
    io=f"data/{file_name}",
    sheet_name="all",
    skiprows=[0,1],
    header=None
)

flt_data.columns = columns
flt_data["Plan Nr."] = flt_data["Plan Nr."].map(flt_object_mapper)
flt_data["Ebene"] = flt_data["Ebene"].astype(str).map(level_mapper)

# Add origin
flt_data["origin"] = file_name
flt_data["aks"] = flt_data["Plan Nr."].astype(str) + " " + flt_data["Ebene"] + flt_data["Modul"].astype(str) + flt_data["Raumnummer"].astype(str) + "." + flt_data["Türnummer"].astype(str)

flt_data = clean_df(flt_data, "flt")

flt_data.head(5)

,flt__laufende_nummer,flt__plan_nr,flt__bauteil,flt__ebene,flt__modul,flt__raumnummer,flt__tuernummer,flt__flucht_moeglich_ja,flt__flucht_moeglich_nein,flt__blindzylinder/rosette/abdeckung,flt__zylinder,flt__panikschloss,flt__versperrbar_ja,flt__versperrbar_nein,flt__nottaster_ja,flt__nottaster_nein,flt__maengel_art,flt__origin,flt__aks
0,1.0,420,A,00,A,3108,01,X,NaN,NaN,X,X,X,NaN,NaN,X,NaN,Sisando_FLT_Obj420.xlsx,420 00A3108.01
1,2.0,420,A,00,NaN,5206,05,NaN,X,NaN,X,NaN,X,NaN,NaN,X,"Panikschloss fehlt (Außentür, Offline-Beschlag?)",Sisando_FLT_Obj420.xlsx,420 00nan5206.05
2,3.0,420,A,00,NaN,4401,07,X,NaN,NaN,NaN,NaN,NaN,X,NaN,X,NaN,Sisando_FLT_Obj420.xlsx,420 00nan4401.07
3,4.0,420,A,00,NaN,2601,06,X,NaN,NaN,X,NaN,X,NaN,NaN,X,NaN,Sisando_FLT_Obj420.xlsx,420 00nan2601.06
4,5.0,420,A,00,NaN,2601,04,X,NaN,NaN,X,X,X,NaN,NaN,X,NaN,Sisando_FLT_Obj420.xlsx,420 00nan2601.04


## CAD Database

* Die Spalten B bis F ergeben die AKS-Nummer (``GAR-TURNUMMER-BAUTEIL``, ``GAR-TURNUMMER-EBENE``, ``GAR-TURNUMMER-MODUL``, ``GAR-TURNUMMER-AKS-NR``, ``GAR-TURNUMMER-NUMMER``)
* Fluchttürnummer in Spalte H (``GAR-FLUCHTTUERNUMMER``)

In [6]:
file_name = "134_gesamt_20241212.xlsx"

cad_data = read_excel(
    io = f"data/{file_name}"
)

cad_data["origin"] = file_name
cad_data["GAR-TURNUMMER-EBENE"] = cad_data["GAR-TURNUMMER-EBENE"].astype(str).map(level_mapper)
cad_data["GAR-TURNUMMER-NUMMER"] = cad_data["GAR-TURNUMMER-NUMMER"].map(cad_number_mapper)
cad_data["aks"] = cad_data["GAR-TURNUMMER-BAUTEIL"].astype(str) + " " + cad_data["GAR-TURNUMMER-EBENE"].astype(str) + cad_data["GAR-TURNUMMER-MODUL"].astype(str) + cad_data["GAR-TURNUMMER-AKS-NR"].astype(str) + "." + cad_data["GAR-TURNUMMER-NUMMER"].astype(str)

cad_data = clean_df(cad_data, "cad")

cad_data.head()

,cad__name,cad__gar-turnummer-bauteil,cad__gar-turnummer-ebene,cad__gar-turnummer-modul,cad__gar-turnummer-aks-nr,cad__gar-turnummer-nummer,cad__gar-turnummer-alt,cad__gar-fluchttuernummer,cad__gar-tur-breite,cad__gar-tur-hohe,cad__gar-bsk,cad__gar-hel,cad__gar-sec,cad__gar-zusatz-1,cad__origin,cad__aks
0,GAR-61-Turnummer-2013,134,00,A,641,01,005A,NaN,NaN,NaN,NEIN,---,---,---,134_gesamt_20241212.xlsx,134 00A641.01
1,GAR-61-Turnummer-2013,134,00,A,710,01,---,NaN,NaN,NaN,NEIN,---,---,---,134_gesamt_20241212.xlsx,134 00A710.01
2,GAR-61-Turnummer-2013,134,00,A,711,01,---,NaN,NaN,NaN,NEIN,---,---,---,134_gesamt_20241212.xlsx,134 00A711.01
3,GAR-61-Turnummer-2013,134,00,A,713,01,---,NaN,NaN,NaN,NEIN,---,---,---,134_gesamt_20241212.xlsx,134 00A713.01
4,GAR-61-Turnummer-2013,134,00,A,722,01,0A0722.01,NaN,85,210,NEIN,---,---,---,134_gesamt_20241212.xlsx,134 00A722.01


## Filemaker Database

In [7]:
file_name = "BSTüren_Filemaker_11.04.22.xlsx"

filemaker_data = read_excel(
    io=f"data/{file_name}",
    sheet_name="Gesamt"
)

filemaker_data = filemaker_data.loc[filemaker_data["Bauteil"]=="420"]

filemaker_data["Ebene"] = filemaker_data["Ebene"].map(level_mapper)
filemaker_data["aks"] = filemaker_data["Bauteil"].astype(str) + " " + filemaker_data["Ebene"] + filemaker_data["BrandmelderNR.:"].astype(str)

filemaker_data = clean_df(df=filemaker_data, origin="fm")

filemaker_data.head()

,fm__bauteil,fm__ebene,fm__topnr,fm__brandmelderkette,fm__brandmeldernr,fm__erfasst_von,fm__tuer_12flg,fm__zugaengigkeit,fm__ok,fm__hersteller,...,fm__nettprice,fm__suanbpos,fm__nettacount,fm__tax,fm__gesuanbot,fm__schadensmeldung,fm__aufnahme,fm__kleinregie_summe,fm__zyklus,fm__aks
4594,420,00,78.0,021,B2605.01,Block,1,NaN,2.0,FRITSCHER,...,NaN,NaN,5651677.5,1130335.5,6782013,NaN,744,10,04,420 00B2605.01
4595,420,00,79.0,018A,B1607.03,Block,1,NaN,2.0,FRITSCHER,...,NaN,NaN,5651677.5,1130335.5,6782013,NaN,744,10,04,420 00B1607.03
4596,420,00,80.0,006,B1607.01,Block,2 FLG. Li,NaN,2.0,FRITSCHER,...,NaN,NaN,5651677.5,1130335.5,6782013,Quellstreifen bei allen alten Türen die neu la...,744,10,04,420 00B1607.01
4597,420,00,81.0,\nBüro,B1605.02,Block,1 FLG. Re,NaN,1.0,NOVOFERM,...,NaN,NaN,5651677.5,1130335.5,6782013,NaN,744,10,04,420 00B1605.02
4598,420,00,82.0,NaN,012,Block,1 FLG. Li,Kollektor,2.0,FRITSCHER,...,1240.0,1240.0,5651677.5,1130335.5,6782013,NaN,744,10,04,420 00012


## Integration of data